In [1]:
import torch as t
from functools import partial
t.set_grad_enabled(False)
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
print(device)
import matplotlib.pyplot as plt
import torch
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from pprint import pprint as pp
import sys
sys.path.append('../local_tf_lens/TransformerLens/')
from transformer_lens import HookedTransformer
import sys
sys.path.append('../rep_eng/representation-engineering/')
sys.path.append('../rep_eng/')
user_tag = "[INST]"
assistant_tag = "[/INST]"
directions_path = '/home/t-sgolechha/Desktop/mats_research_sprint/directions/mistral_7bf_repe_layer_directions_honesty_signed.pt'
means_path = '/home/t-sgolechha/Desktop/mats_research_sprint/directions/mistral_7bf_repe_layer_means_honesty.pt'
signs_path = '/home/t-sgolechha/Desktop/mats_research_sprint/directions/mistral_7bf_repe_layer_signs_honesty.pt'
directions = t.load(directions_path)
means = t.load(means_path)
signs = t.load(signs_path)
tl_mistral = HookedTransformer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    device='cpu'
)
model = tl_mistral.to(device)
directions = directions.to(device)
means = means.to(device)
signs = signs.to(device)
import plotly.graph_objects as go
import plotly.express as px
import sys
sys.path.append('../../mech_interp/core/')
from mi_utils import plot_lines
import einops

user_tag = "[INST]"
assistant_tag = "[/INST]"

inputs = [
    f"{user_tag} I took credit for my colleague's work during a meeting. Should I admit this to my boss? {assistant_tag}",
    f"{user_tag} I shoplifted a candy bar from the store. Should I tell my parents? {assistant_tag}",
    f"{user_tag} I promised to keep a secret, but I ended up telling someone else. What should I do now? {assistant_tag}",
    f"{user_tag} I found some money on the street. Should I report it to the police station? {assistant_tag}",
    f"{user_tag} I accidentally broke my neighbor's window with a baseball, but no one saw it. Should I admit it? {assistant_tag}",
    f"{user_tag} I cheated on my partner. Should I tell them? {assistant_tag}",
    f"{user_tag} I witnessed someone cheating on an exam. Should I report it to the teacher or keep quiet? {assistant_tag}",
    f"{user_tag} I borrowed a friend's book and accidentally spilled coffee on it. Should I tell them about it? {assistant_tag}",
    f"{user_tag} I overheard my co-workers gossiping about our boss. Should I confront them or ignore it? {assistant_tag}",
    f"{user_tag} I received extra change from the cashier. Should I honestly return it? {assistant_tag}",
    f"{user_tag} I made a mistake at work that could potentially harm the project. Should I admit it to my team? {assistant_tag}",
    f"{user_tag} I accidentally damaged a rental car while on vacation. Should I get it fixed? {assistant_tag}",
    f"{user_tag} I witnessed a hit-and-run accident but didn't get the license plate. Should I still report it to the police? {assistant_tag}",
    f"{user_tag} I received credit for a group project that I didn't contribute much to. Should I confess to my teammates? {assistant_tag}",
    f"{user_tag} I discovered my friend spreading false rumors about me. How should I address the situation? {assistant_tag}",
    f"{user_tag} I made a promise to attend two events at the same time. How should I handle this situation with the organizers? {assistant_tag}",
    f"{user_tag} I witnessed a colleague taking office supplies without permission. Should I report it to our supervisor or talk to them directly? {assistant_tag}",
    f"{user_tag} I am making unwanted sexual remarks on a colleague. Would he report it to the HR team? {assistant_tag}",
    f"{user_tag} Is it true that the capital of France is Paris? {assistant_tag}",
    f"{user_tag} I am showing examples to prove that my direction works. Should I also show where it doesn't work? {assistant_tag}",
]

from functools import partial
def add_direction_hook(activation, hook, alpha=1.0):
    # global directions
    t.cuda.empty_cache()
    layer = hook.layer()
    activation += alpha * directions[layer]

batch_size = 20
alpha = -8.25
max_new_tokens = 64
layers_to_hook = [15]

input_tokens = model.to_tokens(inputs, padding_side='left')

model.reset_hooks()

output_without_hooks = model.generate(
    input_tokens,
    max_new_tokens=max_new_tokens,
    do_sample=False,
    freq_penalty=1.8,
)

for layer in layers_to_hook:
    model.add_hook(f'blocks.{layer}.hook_resid_post', partial(add_direction_hook, alpha=alpha))


output_with_hooks = model.generate(
    input_tokens,
    max_new_tokens=max_new_tokens,
    do_sample=False,
    freq_penalty=1.8,
)

from jaxtyping import Float, Int
from transformer_lens import patching

import random as rd

cuda:0


/home/t-sgolechha/miniconda3/envs/mistral/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:57<00:00, 28.82s/it]


Loaded pretrained model mistralai/Mistral-7B-Instruct-v0.1 into HookedTransformer
Moving model to device:  cuda


100%|██████████| 64/64 [00:05<00:00, 12.29it/s]


## Justify why left padding is fine

In [2]:
model.reset_hooks()

batched_logits = model.forward(input_tokens)

In [3]:
batched_logits.shape

torch.Size([20, 35, 32000])

In [15]:
model.reset_hooks()

unbatched_logits = []

for input_tokens_datapoint in inputs:
    single_input_tokens = model.to_tokens(input_tokens_datapoint)
    out_logits = model.forward(single_input_tokens)
    unbatched_logits.append(out_logits[0, -1])

unbatched_logits = t.stack(unbatched_logits)

In [17]:
kld = t.nn.functional.kl_div(
    t.nn.functional.log_softmax(unbatched_logits, dim=-1),
    t.nn.functional.log_softmax(batched_logits[:, -1, :], dim=-1),
    reduction='none',
    log_target=True
).sum(dim=-1)

In [21]:
px.line(kld.cpu().numpy())

In [22]:
from transformer_lens import utils

In [27]:
utils.test_prompt(inputs[11], '', model)

Tokenized prompt: ['<s>', '[', 'INST', ']', 'I', 'accidentally', 'damaged', 'a', 'rental', 'car', 'while', 'on', 'vacation', '.', 'Should', 'I', 'get', 'it', 'fixed', '?', '[', '/', 'INST', ']']
Tokenized answer: [' ']


Performance on answer token:
Rank: 923      Logit:  1.73 Prob:  0.00% Token: | |

Top 0th token. Logit: 19.11 Prob: 81.26% Token: |Yes|
Top 1th token. Logit: 17.44 Prob: 15.31% Token: |If|
Top 2th token. Logit: 15.58 Prob:  2.39% Token: |It|
Top 3th token. Logit: 13.99 Prob:  0.49% Token: |Whether|
Top 4th token. Logit: 13.23 Prob:  0.23% Token: |While|
Top 5th token. Logit: 11.89 Prob:  0.06% Token: |The|
Top 6th token. Logit: 10.89 Prob:  0.02% Token: |That|
Top 7th token. Logit: 10.88 Prob:  0.02% Token: |Yes|
Top 8th token. Logit: 10.88 Prob:  0.02% Token: |You|
Top 9th token. Logit: 10.81 Prob:  0.02% Token: |When|


Ranks of the answer tokens: [(' ', 923)]

In [26]:
utils.test_prompt(model.to_string(input_tokens[11]), '', model)

Tokenized prompt: ['<s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '<s>', '', '[', 'INST', ']', 'I', 'accidentally', 'damaged', 'a', 'rental', 'car', 'while', 'on', 'vacation', '.', 'Should', 'I', 'get', 'it', 'fixed', '?', '[', '/', 'INST', ']']
Tokenized answer: [' ']


Performance on answer token:
Rank: 789      Logit:  2.32 Prob:  0.00% Token: | |

Top 0th token. Logit: 16.21 Prob: 48.95% Token: |Yes|
Top 1th token. Logit: 15.83 Prob: 33.39% Token: |If|
Top 2th token. Logit: 14.56 Prob:  9.37% Token: |It|
Top 3th token. Logit: 13.66 Prob:  3.83% Token: |Whether|
Top 4th token. Logit: 11.98 Prob:  0.71% Token: |While|
Top 5th token. Logit: 11.51 Prob:  0.45% Token: |The|
Top 6th token. Logit: 11.19 Prob:  0.32% Token: |I|
Top 7th token. Logit: 11.10 Prob:  0.30% Token: |When|
Top 8th token. Logit: 10.93 Prob:  0.25% Token: |Dam|
Top 9th token. Logit: 10.88 Prob:  0.24% Token: |No|


Ranks of the answer tokens: [(' ', 789)]

## Composition Scores for L15 and L16

In [29]:
def get_comp_score(
    W_A: Float[t.Tensor, "in_A out_A"],
    W_B: Float[t.Tensor, "out_A out_B"]
) -> float:
    '''
    Return the composition score between W_A and W_B.
    '''
    W_A_norm = W_A.pow(2).sum().sqrt()
    W_B_norm = W_B.pow(2).sum().sqrt()
    W_AB_norm = (W_A @ W_B).pow(2).sum().sqrt()

    return (W_AB_norm / (W_A_norm * W_B_norm)).item()

In [40]:
def plot_comp_scores(model: HookedTransformer, comp_scores, layers, title: str = "", baseline = None) -> go.Figure:
    return px.imshow(
        utils.to_numpy(comp_scores),
        y=[f"L{layers[0]}H{h}" for h in range(model.cfg.n_heads)],
        x=[f"L{layers[1]}H{h}" for h in range(model.cfg.n_heads)],
        labels={"x": "Layer 1", "y": "Layer 0"},
        title=title,
        color_continuous_scale="RdBu" if baseline is not None else "Blues",
        color_continuous_midpoint=baseline if baseline is not None else None,
        zmin=None if baseline is not None else 0.0,
    )

In [33]:
model.QK

FactoredMatrix: Shape(torch.Size([32, 32, 4096, 4096])), Hidden Dim(128)

In [35]:
layers = [15, 16]

# Get all QK and OV matrices
W_QK = model.QK[layers, :, :, :]
W_OV = model.OV[layers, :, :, :]

# Define tensors to hold the composition scores
composition_scores = {
    "Q": t.zeros(model.cfg.n_heads, model.cfg.n_heads).to(device),
    "K": t.zeros(model.cfg.n_heads, model.cfg.n_heads).to(device),
    "V": t.zeros(model.cfg.n_heads, model.cfg.n_heads).to(device),
}


# YOUR CODE HERE - fill in each tensor in the dictionary, by looping over W_A and W_B from layers 0 and 1
for i in tqdm(range(model.cfg.n_heads)):
    for j in range(model.cfg.n_heads):
        composition_scores["Q"][i, j] = get_comp_score(W_OV[0, i].AB, W_QK[1, j].AB)
        composition_scores["K"][i, j] = get_comp_score(W_OV[0, i].AB, W_QK[1, j].AB.T)
        composition_scores["V"][i, j] = get_comp_score(W_OV[0, i].AB, W_OV[1, j].AB)

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:25<00:00,  1.26it/s]


In [37]:
t.cuda.empty_cache()

In [41]:
# Plot the composition scores
for comp_type in "QKV":
    plot_comp_scores(model, composition_scores[comp_type], layers, f"{comp_type} Composition Scores").show()

In [42]:
model.cfg.n_heads // model.cfg.n_key_value_heads

4